# Quadruped Navigation/Motion Control with Text Commands


In [1]:
import torch
import pybullet as p
import pybullet_data
import time

Start Pybullet Client


In [ ]:
physicsClient = p.connect(p.GUI)
p.setAdditionalSearchPath(pybullet_data.getDataPath())
p.setGravity(0, 0, -9.81)   # set gravity

# Load models
planeID = p.loadURDF('plane.urdf')

init_pos = [0, 0, 0.005]
init_orientation = p.getQuaternionFromEuler([0, 0, 0])
robotID = p.loadURDF('quadruped/quadruped.urdf', basePosition=init_pos, baseOrientation=init_orientation)

In [3]:
# Disconnect from client
p.disconnect()